In [11]:
import pandas as pd
import os
current_path = os.path.join(os.getcwd(), 'samples.xlsx')


In [12]:
csv = pd.read_excel(current_path)
csv.head()

,newspaper1,newspaper2,magazine,pack
0,13,53,103,5
1,45,17,30,16
2,22,38,53,14
3,18,33,121,11
4,36,9,26,14


In [13]:
csv.describe()

,newspaper1,newspaper2,magazine,pack
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,22.519000,33.281000,45.854000,11.342000
std,11.778726,17.473104,25.767511,5.652343
min,5.000000,7.000000,10.000000,2.000000
25%,14.000000,21.000000,28.000000,7.000000
50%,20.000000,30.000000,40.000000,10.000000
75%,27.000000,41.000000,57.000000,14.000000
max,92.000000,175.000000,237.000000,42.000000


## Constantes:

desconto_pacote : 8% = 0.08 (1 - 0.08 = 0.92, valor inserido direto na FO)

cj, 𝑐𝑚: Custo de compra do jornal mais caro e da revista.

v𝑗, v𝑚: Preço de venda do jornal mais caro e da revista.

D: Demanda incerta = Cenários D.

𝑟𝑗: Fração do custo de retorno recuperado para os jornais (valor inserido direto na FO).

B: Orçamento disponível.

S: Capacidade total da prateleira.

𝑠𝑗, 𝑠𝑚: Espaço ocupado por um jornal e por uma revista na prateleira.

In [ ]:
D = csv.values
produtos = ['j1', 'j2', 'magazine']

orcamento = 1000
capacidade = 500
espaco = {"j1": 1, "j2": 1, "mag": 1.5}

custo_j = 1
custo_m = 4.0

venda_j = 1.5
venda_m = 5.0

## Variáveis de decisão:

### - Estágio 1

compra_j1 : Quantidade de jornal 1 comprada.

compra_j2 : Quantidade de jornal 2 comprada.

compra_magazine : Quantidade de revistas compradas.

### - Estágio 2

venda_j1 : Quantidade de jornal 1 vendido.

venda_j2 : Quantidade de jornal 2 vendido.

venda_magazine : Quantidade de revistas vendidas.

venda_pacote : Quantidade de pacotes vendidos.

retorno_j1 = Quantidade de jornal 1 não vendida e retornada.

retorno_j2 = Quantidade de jornal 2 não vendida e retornada.

### Func Obj

Lucro = Receita + Reembolso por devolução − Custo de compra

$$

\max \sum_{D} p(D) \Bigg[ (0.9 \cdot v_{j}) \cdot \textbf{venda\_j1}^D + v_{j} \cdot \textbf{venda\_j2}^D + v_m \cdot \textbf{venda\_magazine}^D + (0.92 \cdot (v_{j} + v_m)) \cdot \textbf{venda\_pacote}^D \\
 + (0.1 \cdot (0.9 \cdot c_{j})) \cdot \textbf{retorno\_j1}^D + (0.4 \cdot c_{j}) \cdot \textbf{retorno\_j2}^D \Bigg] \\ 
 - \Bigg[ (0.9 \cdot c_{j}) \cdot \textbf{compra\_j1} + c_{j} \cdot \textbf{compra\_j2} + c_m \cdot \textbf{compra\_magazine} \Bigg]

$$

In [ ]:
from gurobipy import Model, GRB

model = Model("model")

compra = model.addVars(produtos, vtype=GRB.INTEGER, name="compra")
venda = model.addVars(produtos + ['pack'], vtype=GRB.INTEGER, name="venda")
retorno = model.addVars(['j1', 'j2'], vtype=GRB.INTEGER, name="retorno")

# FO:
lucro_esperado = (
    0.9*venda_j * venda['j1'] + venda_j * venda['j2'] + venda_m * venda['magazine'] + 0.92*(venda_j + venda_m) * venda['pack'] +
    0.1 * 0.9*venda_j * retorno['j1'] + 0.4*custo_j * retorno['j2'] -
    (0.9 * custo_j * compra['j1'] + custo_j * compra['j2'] + custo_m * compra['magazine'])
)
model.setObjective(lucro_esperado, GRB.MAXIMIZE)

# Restricoes:
model.addConstr(0.9*custo_j * compra['j1'] + custo_j * compra['j2'] + custo_m * compra['magazine'] <= orcamento)
model.addConstr(espaco["j1"] * (compra['j1'] + compra['j2']) + espaco["mag"] * compra['magazine'] <= capacidade)
model.addConstr(venda['j1'] + retorno['j1'] <= compra['j1'])
model.addConstr(venda['j2'] + retorno['j2'] + venda['pack'] <= compra['j2'])
model.addConstr(venda['magazine'] + venda['pack'] <= compra['magazine'])
model.addConstr(venda['j2'] + venda['pack'] <= compra['j2'])

model.write("model.lp")

Restricted license - for non-production use only - expires 2026-11-23


In [17]:
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 9 columns and 19 nonzeros
Model fingerprint: 0xcaba0862
Variable types: 0 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [9e-01, 4e+00]
  Objective range  [1e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 4 rows and 6 columns
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)
Found heuristic solution: objective 225.0000000

Root relaxation: objective 3.000000e+02, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | In

In [18]:
model.getVars()

[<gurobi.Var compra[j1] (value -0.0)>,
 <gurobi.Var compra[j2] (value 200.0)>,
 <gurobi.Var compra[magazine] (value 200.0)>,
 <gurobi.Var venda[j1] (value 0.0)>,
 <gurobi.Var venda[j2] (value 200.0)>,
 <gurobi.Var venda[magazine] (value 200.0)>,
 <gurobi.Var venda[pack] (value -0.0)>,
 <gurobi.Var retorno[j1] (value -0.0)>,
 <gurobi.Var retorno[j2] (value -0.0)>]